Implements algorithm 3 (ModMod) from Algorithms for "Approx Min of the Difference Between Submodular Fncs with Applications."

For prob_ssm

In [1]:
import torch
from torch import distributions as dist

import itertools
import pickle
import operator

import random
import math
import numpy as np
import pandas as pd
import scipy

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('white')
sns.set_context('paper')
# Plot adjustments:
plt.rcParams.update({'ytick.labelsize': 15})
plt.rcParams.update({'xtick.labelsize': 15})
plt.rcParams.update({'axes.labelsize': 35})
plt.rcParams.update({'legend.fontsize': 30})
plt.rcParams.update({'axes.titlesize': 16})

from gptorch import kernels, models

import helpers, opt

In [2]:
with open('../inputs/phoq.pkl', 'rb') as f:
    t = pickle.load(f)

X = t[0] # one-hot encoding of X
T = t[1] # tokenized encoding of X
y = t[2].values

In [ ]:
np.random.seed(1)
rand_inds = np.random.choice(len(X), 100, replace=True) # generate random indices for 100 X's to sample from
X_train = X[rand_inds]
y_train = y[rand_inds]
X_test = X
y_true = y

dic = helpers.get_predictions(X_train, y_train, X_test, its=500)

In [ ]:
ground = [(aa, i) for aa in 'ARNDCQEGHILKMFPSTWYV' for i in range(4)]

In [ ]:
np.random.seed(4195)
center = np.random.choice(len(ground), 35, replace=False)
center = [ground[c] for c in center]

In [ ]:
X, h = opt.mod_mod(ground, opt.obj_LHS, opt.obj_RHS, center, fn_args=(4, dic), g_args=(4, dic, 100))

In [3]:
def sampler_full_obj(lib, mu, K, tau, its=10, n=100):
    """ Takes in a library (list of tuples with aa and positions),
    full mean (a dictionary mapping sequence, a string, to its mean)
    and covariance, threshold, number of iterations, and
    batch size. 
    
    Samples from full objective, and returns the number sampled that is 
    greater than tau (threshold). """
    
    num_greater = []
    lib = np.array(helpers.seqs_from_set(lib, 4)) # extract variants from library
    
    for _ in range(its):
        rand_inds = np.random.choice(len(lib), n, replace=True) # generate random indices
        rand_inds = np.unique(rand_inds) # remove duplicates
        chosen = lib[rand_inds]
        means_chosen = np.array([mu[key] for key in mu if key in chosen])
        inds = np.array([list(mu.values()).index(m) for m in means_chosen])
        sample = np.random.multivariate_normal(means_chosen, K[inds, inds]) # (n, ) vector
        num_greater.append(np.sum(s > tau))
    
    return num_greater

In [5]:
np.random.seed(1)
rand_inds = np.random.choice(len(X), 100, replace=True) # generate random indices for 100 X's to sample from
X_train = X[rand_inds]
y_train = y[rand_inds]
X_test = X
y_true = y

dic, means = helpers.get_predictions(X_train, y_train, X_test, its=500)

Iteration 500 of 500	NLML: 37.3695	sn: 0.179451	

In [7]:
ke = kernels.MaternKernel()
cov = ke(torch.Tensor(X), torch.Tensor(X))

RuntimeError: $ Torch: not enough memory: you tried to allocate 95GB. Buy new RAM! at ..\src\TH\THGeneral.c:218

In [ ]:
lib = [('F', 3), ('V', 1), ('P', 2), ('A', 0), ('R', 0), ('Q', 3)]
tau = helpers.get_seed(dic)
sampler_full_obj(lib, means, cov, tau)